# Read in Poore and Nemecek 2018 data

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

The FixOurFood Dashboard utilizes FAO food supply data to estimate CO2e emissions from consumed food items in the UK. Estimations of emissions per unit mass of consumed food come from Life Cycle Assestment studies (LCA) which aggregate the emissions associated to all the steps of production, management and consumption.

Poore And Nemecek (2018) [1] present a comprehensive list of LCA estimated CO2e emissions for a range of food items, wichi we match here to FAO food items.
Data is open access and can be downloaded from here:
https://www.science.org/doi/suppl/10.1126/science.aaq0216/suppl_file/aaq0216_datas2.xls

This notebook will read the values from this file and assign them to the corresponding FAO `Item Codes`.
Every item will also be assigned to one of 12 food groups:

- 0 : Ruminant meat
- 1 : Other meat
- 2 : Egg
- 3 : Dairy
- 4 : Cereals, starch
- 5 : Pulses
- 6 : Sugar, veg oil
- 7 : Nuts, seeds
- 8 : Vegetables, fruits
- 9 : Other
- 10 : Fish, seafood
- 11 : Non-food

For some food items, closes equivalents will be used, while some other cases require special attention.
Each case will be described in detail.
The output will be an array containing:

- code : FAO Item Code
- name : FAO Item
- group : Food Group Code
- group_id : Food Group
- mean_emissions : GHGE intensity global mean (kg CO2e/kg)

In [2]:
datafile = "../../data/food/aaq0216_datas2.xls"
fbs = pd.read_csv('../../data/food/food_supply_data.csv')

item_dict = {}
item_codes = np.unique(fbs['Item Code'])
for code in item_codes:
    name = fbs[fbs['Item Code'] == code]['Item'].iloc[0]
    item_dict[name] = code
    
for name, code in item_dict.items():
    print(code, name)

2501 Population
2511 Wheat and products
2513 Barley and products
2514 Maize and products
2515 Rye and products
2516 Oats
2520 Cereals, Other
2531 Potatoes and products
2534 Roots, Other
2535 Yams
2542 Sugar (Raw Equivalent)
2543 Sweeteners, Other
2546 Beans
2547 Peas
2549 Pulses, Other and products
2551 Nuts and products
2555 Soyabeans
2556 Groundnuts (Shelled Eq)
2558 Rape and Mustardseed
2560 Coconuts - Incl Copra
2561 Sesame seed
2563 Olives (including preserved)
2570 Oilcrops, Other
2571 Soyabean Oil
2572 Groundnut Oil
2573 Sunflowerseed Oil
2574 Rape and Mustard Oil
2575 Cottonseed Oil
2576 Palmkernel Oil
2577 Palm Oil
2578 Coconut Oil
2579 Sesameseed Oil
2580 Olive Oil
2582 Maize Germ Oil
2586 Oilcrops Oil, Other
2601 Tomatoes and products
2602 Onions
2605 Vegetables, other
2611 Oranges, Mandarines
2612 Lemons, Limes and products
2613 Grapefruit and products
2614 Citrus, Other
2615 Bananas
2616 Plantains
2617 Apples and products
2618 Pineapples and products
2619 Dates
2620 Grapes

In [3]:
data = pd.read_excel(datafile, sheet_name='Results - Retail Weight', header=2, nrows=43)
data = data.rename(columns={"Mean.1":"GHGE intensity global mean (kg CO2e/kg)"})
data[['Product','GHGE intensity global mean (kg CO2e/kg)']]

,Product,GHGE intensity global mean (kg CO2e/kg)
0,Wheat & Rye (Bread),1.57
1,Maize (Meal),1.70
2,Barley (Beer),1.18
3,Oatmeal,2.48
4,Rice,4.45
5,Potatoes,0.46
6,Cassava,1.32
7,Cane Sugar,3.20
8,Beet Sugar,1.81
9,Other Pulses,1.79


# Direct matchs

A series of items have direct counterparts which we use to assign mean GHGE values

| FAO                           | PN18                  |GHGE intensity (kg CO2e/kg)| Food Group |
|-------------------------------|-----------------------|---------------------------|------------|
|Apples and products            |Apples                 |0.40                       | 8          |
|Bananas                        |Bananas                |0.90                       | 8          |
|Beans                          |Beans & Pulses         |1.80                       | 5          |
|Beer                           |Beer                   |1.20                       | 9          |
|Beverages, Alcoholic           |Wine                   |1.80                       | 9          |
|Beverages, Fermented           |Beer                   |1.20                       | 9          |
|Bovine Meat                    |Beef                   |70.50                      | 0          |
|Cassava and products           |Cassava and Other Roots|1.30                       | 8          |
|Citrus, other                  |Citrus Fruit           |0.40                       | 8          |
|Coffe and products             |Coffe                  |28.50                      | 9          |
|Dates                          |Other Fruit            |1.10                       | 8          |
|Eggs                           |Eggs                   |4.70                       | 2          |
|Fruits, Other                  |Other Fruit            |1.10                       | 8          |
|Grapefruit and products        |Citrus Fruit           |0.40                       | 8          |
|Grapes and products (excl wine)|Berries and Grapes     |1.50                       | 8          |
|Groundnuts (Shelled Eq)        |Groundnuts             |3.20                       | 7          |
|Lemons, Limes and products     |Citrus Fruit           |0.40                       | 8          |
|Maize and products             |Maize Meal             |1.70                       | 4          |
|Milk - Excluding Butter        |Milk                   |3.20                       | 3          |
|Mutton & Goat Meat             |Mutton & Goat Meat     |39.70                      | 0          |
|Nuts and products              |Nuts                   |0.40                       | 7          |
|Oilcrops Oil, Other            |Rapeseed (Oil)         |3.80                       | 6          |
|Olive Oil                      |Olives (Oil)           |5.40                       | 6          |
|Onions                         |Onions and Leeks       |0.50                       | 8          |
|Oranges, Mandarines            |Citrus Fruit           |0.40                       | 8          |
|Palm Oil                       |Palm (Oil)             |7.30                       | 6          |
|Palmkernel Oil                 |Palm (Oil)             |7.30                       | 6          |
|Peas                           |Peas                   |1.00                       | 5          |
|Pigmeat                        |Pig Meat               |12.30                      | 1          |
|Pineapples and products        |Other Fruit            |1.10                       | 8          |
|Potatoes and products          |Potatoes               |0.50                       | 4          |
|Poultry meat                   |Poultry Meat           |9.90                       | 1          |
|Pulses, Other and products     |Beans & Pulses         |1.80                       | 5          |
|Rape and Mustard Oil           |Rapeseed (Oil)         |3.80                       | 6          |
|Rice and products              |Rice                   |4.50                       | 4          |
|Ricebran Oil                   |Sunflower (Oil)        |3.60                       | 6          |
|Roots, Other                   |Root Vegetables        |0.40                       | 4          |
|Soyabean Oil                   |Soybeans (Oil)         |6.30                       | 6          |
|Soyabeans                      |Beans & Pulses         |1.80                       | 5          |
|Sunflowerseed Oil              |Sunflower (Oil)        |3.60                       | 6          |
|Sweet potatoes                 |Cassava and Other Roots|1.30                       | 4          |
|Tea (Including mate)           |Tea                    |21.00                      | 9          |
|Tomatoes and products          |Tomatoes               |2.10                       | 8          |
|Vegetables, Other              |Other Vegetables       |0.50                       | 8          |
|Wine                           |Wine                   |1.80                       | 9          |
|Yams                           |Cassava and Other Roots|1.30                       | 4          |

# Closest matches

For some items, close matches are used from PN18

| FAO                    | PN18                  |GHGE intensity (kg CO2e/kg)| Food Group |
|------------------------|-----------------------|---------------------------|------------|
|Aquatic Plants          |Brassicas              | 0.50                      | 8          |
|Cephalopods             |Crustaceans (Farmed)   | 26.90                     | 10         |
|Cloves                  |Nuts                   | 0.40                      | 7          |
|Cocoa Beans and products|Chocolate              | 46.70                     | 9          |
|Coconut Oil             |Palm (Oil)             | 7.30                      | 6          |
|Coconuts - Incl Copra   |Nuts                   | 0.40                      | 7          |
|Crustaceans             |Crustaceans (Farmed)   | 26.90                     | 10         |
|Groundnut Oil           |Soyabeans (Oil)        | 6.30                      | 6          |
|Maize Germ Oil          |Rapeseed (Oil)         | 3.80                      | 6          |
|Molluscs, Other         |Crustaceans (Farmed)   | 26.90                     | 10         |
|Pimento                 |Other Vegetables       | 0.50                      | 8          |
|Platains                |Bananas                | 0.90                      | 8          |
|Sesameseed Oil          |Rapeseed (Oil)         | 3.80                      | 6          |
|Sugar (Raw Equivalent)  |Sugar (from sugar cane)| 3.20                      | 6          |
|Sugar non-centrifugar   |Sugar (from sugar cane)| 3.20                      | 6          |

# Special cases

## Fish

We use the same value for all the fish entries,
assigning them an estimate of the global average
GHGE for fish. We estimate a global average GHGE
for fish by combining the mean global farmed fish value
from PN18 and our estimate of the caught fish value,
weighting by their respective global fractions (55%
farmed and 45% captured, from [44]). We estimate
the caught fish GHGE value from the global analysis
by [45] who estimate the emissions per kg of live weight
of fish for human consumption to be 2.7 kg CO2e /
kg. We convert this to the edible weight assuming
a conversion ratio based on the range of values for
different types of fish, which span from 37% for tilapia
to 67% for mackerel from [46]. We consider the losses
of 26.7% occuring throughout the supply chain until
retail as stated by PN18 for fish and then add on the
emissions for the later stages of the supply chain up to
retail, using the values given by PN18 for farmed fish of
1.14 kg CO2e / kg retail weight which brings the value
to 9.6 to 11.8 (for tilapia and mackerel respectively),
which we average to 10.7 kg CO2e / kg retail weight.

This then applies for the following entries

| FAO                  |GHGE intensity (kg CO2e/kg)| Food Group |
|----------------------|---------------------------|------------|
|Aquatic Animals, Other|  10.7                     | 10         |
|Demersal Fish         |  10.7                     | 10         |
|Freshwater Fish       |  10.7                     | 10         |
|Marine Fish, Other    |  10.7                     | 10         |
|Meat, Aquatic Mammals |  10.7                     | 10         |
|Pelagic Fish          |  10.7                     | 10         |

## Cereals, Starchy Vegetals

'Oatmeal' value, modified by the reciprocal of
ConvWt Oats = 1.36 from [1] to account for processing.
This results in a GHGE value of 2.48 / 1.36 = 1.84 kg CO2e / kg.

| FAO                |GHGE intensity (kg CO2e/kg)| Food Group |
|--------------------|---------------------------|------------|
|Barley and Products | 1.84                      | 4          |
|Millet and Products | 1.84                      | 4          |
|Oats                | 1.84                      | 4          |
|Rye and Prodcuts    | 1.84                      | 4          |
|Sorghum and Products| 1.84                      | 4          |

## Sugar, Vegetable Oil

'Rapeseed (Oil)' value modified by the reciprocal of
ConvWt Rape = 2.36 from [1] to account for processing.
This results in a GHGE value of 3.77 / 2.36 = 1.61 kg CO2e / kg.

| FAO                |GHGE intensity (kg CO2e/kg)| Food Group |
|--------------------|---------------------------|------------|
|Oilcrops, Other     | 1.61                      | 6          |
|Pepper              | 1.61                      | 9          |
|Rape and Mustardseed| 1.61                      | 6          |
|Sesame Seed         | 1.61                      | 7          |
|Spices, Other       | 1.61                      | 9          |

## Processing modifications

| FAO          |   Comment                                      |GHGE intensity (kg CO2e/kg)| Food Group |
|--------------|------------------------------------------------|---------------------------|------------|
|Palm Kernels  | Palm Oil / ConvWt_Palm_Oil from                | 1.56                      | 6          |
|Sugar beet    | Sugar (from sugar beet) / ConvWt_Beet_Sugar [1]| 0.24                      | 6          |
|Sugar cane    | Sugar (from sugar cane) / ConvWt_Cane_Sugar [1]| 0.32                      | 6          |
|Sunflower Seed| Sunflower (Oil) / ConvWt_Sunflower from        | 0.81                      | 6          |

## Averaged fom multiple sources

| FAO          |   Comment                             |GHGE intensity (kg CO2e/kg)| Food Group |
|--------------|---------------------------------------|---------------------------|------------|
|Butter, Ghee  | Average from [37], [38], [39] and [40]| 8.30                      | 3          |
|Cereals, Other| Quinoa average from [41] and [42]     | 0.95                      | 4          |
|Cream         | Average from [40], [43] and [38]      | 4.89                      | 3          |
|Honey         | Average from [47], [48] and [49]      | 1.46                      | 6          |

## Infant food

For infant food we combine values from the
literature according to the fractional composition of
this category, which comprises all food fed to babies
between 0-12 months. We focus on the majority of
infant food consisting of milk formula, cereal porridge
and prepared food. Given that milk formula may be
fed from birth to 12 months and that solid food such as
prepared lunches and cereal porridge is fed from about
6 months, we assume that milk formula accounts for
the majority of infant food (60%) following with equal
share of prepared and dried food (20% each). For milk
formula we use milk powder as a proxy, obtaining the
GHGE value for milk powder by multiplying the PN18
Milk global average value of 3.2 kg CO 2 e / kg by the
mass ratio from liquid to dry milk of 7.69, to give 24.6
kg CO 2 e / kg. The value of 7.69 comes from following
the approach of [50] who cite data from [38], which they
note agrees well with the average of values given by [40]
of 5.12 and 9.22. For the prepared food use a value of
3.99 kg CO 2 e / kg by averaging the values for prepared
lunch from [51]. For the dried food we calculate dry
porridge numbers from [51]: we use their value of 141 g
CO 2 e per meal of 125 g of porridge containing 31.25g of
dry porridge. i.e. (141 g CO 2 e / 125 g meal) / ( 31.25
g dry porridge / 125 g meal) = 4.51 kg CO 2 e / kg dry
porridge. Weighting these values by the proportions
gives 60/100 × 24.6 + 20/100 × 3.99 + 20/100 × 4.51 =
16.5 kg CO 2 e / kg.

| FAO        |GHGE intensity (kg CO2e/kg)| Food Group |
|------------|---------------------------|------------|
| Infant food| 16.50                     | 9          |

## Meat, Other

‘Meat, Other’ is a mixture of the less commonly
eaten meats, including snails, rabbits, camels, horses,
game etc. We match it to PN18 item ‘Pig Meat’
to be conservative about the potential high emissions
from this category.

| FAO        |GHGE intensity (kg CO2e/kg)| Food Group |
|------------|---------------------------|------------|
| Meat, Other| 12.30                     | 1          |

## Olives
For Olives, we use the Olive Oil result from PN18,
and convert it using ConvWt Olives = 4.74 from PN18
to convert from their oil value to the olives value, after
subtracting off the non-farm contributions to the oil
from olives calculation of PN18 (1.3 + 0.4 + 0.8 +
0.04 + 0.3) = 2.84. Then we add on the post-farmgate
contributions by approximating olives as tomatoes, so
adding (0.2 + 0.2 + 0.02 + 0.1) = 0.52 for transport,
packaging, retail and losses. Global value for GHGE
changes from 5.4 to (5.4 - 2.84) / 4.74 + 0.52 = 1.06.

| FAO                     |GHGE intensity (kg CO2e/kg)| Food Group |
|-------------------------|---------------------------|------------|
| Olives (Incl. preserved)| 1.06                      | 8          |

## Wheat and Products
We base the ‘Wheat and products’ value on
‘Bread’ from PN18, but undo the weight conversion
from wheat to bread by PN18, thereby retaining
all post-farmgate impacts, which is appropriate if
we assume all human-consumed wheat is provided
in the form of bread or similar (e.g. pasta). We
use into account ConvWt Cereal Flour (×1.3) and
ConvWt Flour Bread (×0.75) from [1], which gives
1.57 / 1.3 / 0.75 = 1.61


| FAO               |GHGE intensity (kg CO2e/kg)| Food Group |
|-------------------|---------------------------|------------|
| Wheat and products| 1.61                      | 4          |

## Excluded

We also exlude some items, either due to low nutritional value, or not being considered foods

| FAO              |GHGE intensity (kg CO2e/kg)| Comment                         | Food Group |
|------------------|---------------------------|---------------------------------|------------|
| Fats, Animal, Raw| 0.00                      | Low value byproduct             | 9          |
| Offals, Edible   | 0.00                      | Low value byproduct             | 1          |
| Sweeteners, Other| 0.00                      | We will exclude it from analysis| 6          |

In [4]:
# Creating the dataframe
food_info_dict = {}

# Ruminant meat
food_info_dict['Bovine Meat'] = [item_dict['Bovine Meat'], 0, 'Ruminant Meat', 70.50]
food_info_dict['Mutton & Goat Meat'] = [item_dict['Mutton & Goat Meat'], 0, 'Ruminant Meat', 39.70]

# Other meat
food_info_dict['Pigmeat'] = [item_dict['Pigmeat'], 1, 'Other Meat', 12.30]
food_info_dict['Poultry Meat'] = [item_dict['Poultry Meat'], 1, 'Other Meat', 9.90]
food_info_dict['Meat, Other'] = [item_dict['Meat, Other'], 1, 'Other Meat', 12.30]
food_info_dict['Offals, Edible'] = [item_dict['Offals, Edible'], 1, 'Other Meat', 0.00]

# Egg
food_info_dict['Eggs'] = [item_dict['Eggs'], 2, 'Egg', 4.7]

# Dairy
food_info_dict['Butter, Ghee'] = [item_dict['Butter, Ghee'], 3, 'Dairy', 8.30]
food_info_dict['Cream'] = [item_dict['Cream'], 3, 'Dairy', 4.89]
food_info_dict['Milk - Excluding Butter'] = [item_dict['Milk - Excluding Butter'], 3, 'Dairy', 3.20]

# Cereals, starch
food_info_dict['Wheat and products'] = [item_dict['Wheat and products'], 4, 'Cereals, starch', 1.64]
food_info_dict['Barley and products'] = [item_dict['Barley and products'], 4, 'Cereals, starch', 1.84]
food_info_dict['Maize and products'] = [item_dict['Maize and products'], 4, 'Cereals, starch', 1.70]
food_info_dict['Rye and products'] = [item_dict['Rye and products'], 4, 'Cereals, starch', 1.84]
food_info_dict['Oats'] = [item_dict['Oats'], 4, 'Cereals, starch', 1.84]
# food_info_dict['Millet and products'] = [item_dict['Millet and products'], 4, 'Cereals, starch', 1.84]
# food_info_dict['Sorghum and products'] = [item_dict['Sorghum and products'], 4, 'Cereals, starch', 1.84]
food_info_dict['Cereals, Other'] = [item_dict['Cereals, Other'], 4, 'Cereals, starch', 0.95]
food_info_dict['Potatoes and products'] = [item_dict['Potatoes and products'],4,'Cereals, starch', 0.50]
# food_info_dict['Cassava and products'] = [item_dict['Cassava and products'], 4, 'Cereals, starch', 1.30]
# food_info_dict['Sweet potatoes'] = [item_dict['Sweet potatoes'], 4, 'Cereals, starch', 1.30]
food_info_dict['Roots, Other'] = [item_dict['Roots, Other'], 4, 'Cereals, starch', 0.40]
food_info_dict['Yams'] = [item_dict['Yams'], 4, 'Cereals, starch', 1.30]
food_info_dict['Rice and Products'] = [item_dict['Rice and Products'], 4, 'Cereals, starch', 4.50]

# Pulses
food_info_dict['Beans'] = [item_dict['Beans'], 5, 'Pulses', 1.80]
food_info_dict['Peas'] = [item_dict['Peas'], 5, 'Pulses', 1.00]
food_info_dict['Pulses, Other and products'] = [item_dict['Pulses, Other and products'], 5, 'Pulses', 1.80]
food_info_dict['Soyabeans'] = [item_dict['Soyabeans'], 5, 'Pulses', 1.80]

# Sugar, veg. oil
# food_info_dict['Sugar cane'] = [item_dict['Sugar cane'], 6, 'Sugar, veg. oil', 0.32]
# food_info_dict['Sugar beet'] = [item_dict['Sugar beet'], 6, 'Sugar, veg. oil', 0.24]
# food_info_dict['Sugar non-centrifugal'] = [item_dict['Sugar non-centrifugal'], 6, 'Sugar, veg. oil', 3.20]
# food_info_dict['Sugar (Raw Equivalent)'] = [item_dict['Sugar (Raw Equivalent)'], 6, 'Sugar, veg. oil', 3.20]
food_info_dict['Sweeteners, Other'] = [item_dict['Sweeteners, Other'], 6, 'Sugar, veg. oil', 0.00]
food_info_dict['Rape and Mustardseed'] = [item_dict['Rape and Mustardseed'], 6, 'Sugar, veg. oil', 1.61]
# food_info_dict['Palm kernels'] = [item_dict['Palm kernels'], 6, 'Sugar, veg. oil', 1.56]
food_info_dict['Oilcrops, Other'] = [item_dict['Oilcrops, Other'], 6, 'Sugar, veg. oil', 1.61]
food_info_dict['Soyabean Oil'] = [item_dict['Soyabean Oil'], 6, 'Sugar, veg. oil', 6.30]
food_info_dict['Groundnut Oil'] = [item_dict['Groundnut Oil'], 6, 'Sugar, veg. oil', 6.30]
food_info_dict['Sunflowerseed Oil'] = [item_dict['Sunflowerseed Oil'], 6, 'Sugar, veg. oil', 3.60]
food_info_dict['Rape and Mustard Oil'] = [item_dict['Rape and Mustard Oil'], 6, 'Sugar, veg. oil', 3.80]
food_info_dict['Palmkernel Oil'] = [item_dict['Palmkernel Oil'], 6, 'Sugar, veg. oil', 7.30]
food_info_dict['Palm Oil'] = [item_dict['Palm Oil'], 6, 'Sugar, veg. oil', 7.30]
food_info_dict['Coconut Oil'] = [item_dict['Coconut Oil'], 6, 'Sugar, veg. oil', 7.30]
food_info_dict['Sesameseed Oil'] = [item_dict['Sesameseed Oil'], 6, 'Sugar, veg. oil', 3.80]
food_info_dict['Olive Oil'] = [item_dict['Olive Oil'], 6, 'Sugar, veg. oil', 5.40]
# food_info_dict['Ricebran Oil'] = [item_dict['Ricebran Oil'], 6, 'Sugar, veg. oil', 3.60]
food_info_dict['Maize Germ Oil'] = [item_dict['Maize Germ Oil'], 6, 'Sugar, veg. oil', 3.80]
food_info_dict['Oilcrops Oil, Other'] = [item_dict['Oilcrops Oil, Other'], 6, 'Sugar, veg. oil', 3.80]
food_info_dict['Honey'] = [item_dict['Honey'], 6, 'Sugar, veg. oil', 1.46]

# Nuts, seeds
food_info_dict['Nuts and products'] = [item_dict['Nuts and products'], 7, 'Nuts, seeds', 0.40]
food_info_dict['Groundnuts (Shelled Eq)'] = [item_dict['Groundnuts (Shelled Eq)'], 7, 'Nuts, seeds', 3.20]
# food_info_dict['Sunflower seed'] = [item_dict['Sunflower seed'], 7, 'Nuts, seeds', 1.81]
food_info_dict['Coconuts - Incl Copra'] = [item_dict['Coconuts - Incl Copra'], 7, 'Nuts, seeds', 0.40]
food_info_dict['Sesame seed'] = [item_dict['Sesame seed'], 7, 'Nuts, seeds', 1.61]

# Vegetables, fruits
food_info_dict['Aquatic Plants'] = [item_dict['Aquatic Plants'], 8, 'Vegetables, fruits', 0.50]
food_info_dict['Olives (including preserved)'] = [item_dict['Olives (including preserved)'], 8, 'Vegetables, fruits', 1.06]
food_info_dict['Tomatoes and products'] = [item_dict['Tomatoes and products'], 8, 'Vegetables, fruits', 2.10]
food_info_dict['Onions'] = [item_dict['Onions'], 8, 'Vegetables, fruits', 0.50]
# food_info_dict['Vegetables, Other'] = [item_dict['Vegetables, Other'], 8, 'Vegetables, fruits', 0.50]
food_info_dict['Oranges, Mandarines'] = [item_dict['Oranges, Mandarines'], 8, 'Vegetables, fruits', 0.40]
food_info_dict['Lemons, Limes and products'] = [item_dict['Lemons, Limes and products'], 8, 'Vegetables, fruits', 0.40]
food_info_dict['Grapefruit and products'] = [item_dict['Grapefruit and products'], 8, 'Vegetables, fruits', 0.40]
food_info_dict['Citrus, Other'] = [item_dict['Citrus, Other'], 8, 'Vegetables, fruits', 0.40]
food_info_dict['Bananas'] = [item_dict['Bananas'], 8, 'Vegetables, fruits', 0.90]
food_info_dict['Plantains'] = [item_dict['Plantains'], 8, 'Vegetables, fruits', 0.90]
food_info_dict['Apples and products'] = [item_dict['Apples and products'], 8, 'Vegetables, fruits', 0.40]
food_info_dict['Pineapples and products'] = [item_dict['Pineapples and products'], 8, 'Vegetables, fruits', 1.10]
food_info_dict['Dates'] = [item_dict['Dates'], 8, 'Vegetables, fruits', 1.10]
food_info_dict['Grapes and products (excl wine)'] = [item_dict['Grapes and products (excl wine)'], 8, 'Vegetables, fruits', 1.50]
# food_info_dict['Fruits, Other'] = [item_dict['Fruits, Other'], 8, 'Vegetables, fruits', 1.10]
food_info_dict['Pimento'] = [item_dict['Pimento'], 8, 'Vegetables, fruits', 0.50]

# Other
food_info_dict['Coffee and products'] = [item_dict['Coffee and products'], 9, 'Other', 28.50]
food_info_dict['Cocoa Beans and products'] = [item_dict['Cocoa Beans and products'], 9, 'Other', 46.70]
food_info_dict['Tea (including mate)'] = [item_dict['Tea (including mate)'], 9, 'Other', 21.00]
food_info_dict['Pepper'] = [item_dict['Pepper'], 9, 'Other', 1.61]
food_info_dict['Cloves'] = [item_dict['Cloves'], 9, 'Other', 0.40]
food_info_dict['Spices, Other'] = [item_dict['Spices, Other'], 9, 'Other', 1.61]
food_info_dict['Wine'] = [item_dict['Wine'], 9, 'Other', 1.80]
food_info_dict['Beer'] = [item_dict['Beer'], 9, 'Other', 1.20]
food_info_dict['Beverages, Fermented'] = [item_dict['Beverages, Fermented'], 9, 'Other', 1.20]
food_info_dict['Beverages, Alcoholic'] = [item_dict['Beverages, Alcoholic'], 9, 'Other', 1.80]
food_info_dict['Infant food'] = [item_dict['Infant food'], 9, 'Other', 16.40]
food_info_dict['Fats, Animals, Raw'] = [item_dict['Fats, Animals, Raw'], 9, 'Other', 0.00]

# Fish, seafood
food_info_dict['Freshwater Fish'] = [item_dict['Freshwater Fish'], 10, 'Fish, seafood', 10.70]
food_info_dict['Demersal Fish'] = [item_dict['Demersal Fish'], 10, 'Fish, seafood', 10.70]
food_info_dict['Pelagic Fish'] = [item_dict['Pelagic Fish'], 10, 'Fish, seafood', 10.70]
food_info_dict['Marine Fish, Other'] = [item_dict['Marine Fish, Other'], 10, 'Fish, seafood', 10.70]
food_info_dict['Crustaceans'] = [item_dict['Crustaceans'], 10, 'Fish, seafood', 26.90]
food_info_dict['Cephalopods'] = [item_dict['Cephalopods'], 10, 'Fish, seafood', 26.90]
food_info_dict['Molluscs, Other'] = [item_dict['Molluscs, Other'], 10, 'Fish, seafood', 26.90]
# food_info_dict['Meat, Aquatic Mammals'] = [item_dict['Meat, Aquatic Mammals'], 10, 'Fish, seafood', 10.70]
food_info_dict['Aquatic Animals, Others'] = [item_dict['Aquatic Animals, Others'], 10, 'Fish, seafood', 10.70]

# Non-food
food_info_dict['Fish, Body Oil'] = [item_dict['Fish, Body Oil'], 11, 'Non-food', 0.00]
food_info_dict['Fish, Liver Oil'] = [item_dict['Fish, Liver Oil'], 11, 'Non-food', 0.00]
# food_info_dict['Alcohol, Non-Food'] = [item_dict['Alcohol, Non-Food'], 11, 'Non-food', 0.00]
food_info_dict['Cottonseed Oil'] = [item_dict['Cottonseed Oil'], 11, 'Non-food', 0.00]
# food_info_dict['Cottonseed'] = [item_dict['Cottonseed'], 11, 'Non-food', 0.00]

food_info = pd.DataFrame.from_dict(food_info_dict, orient='index',
                                   columns = ['code', 'group_id', 'group', 'mean_emissions'])

In [5]:
food_info

,code,group_id,group,mean_emissions
Bovine Meat,2731,0,Ruminant Meat,70.5
Mutton & Goat Meat,2732,0,Ruminant Meat,39.7
Pigmeat,2733,1,Other Meat,12.3
Poultry Meat,2734,1,Other Meat,9.9
"Meat, Other",2735,1,Other Meat,12.3
...,...,...,...,...
"Molluscs, Other",2767,10,"Fish, seafood",26.9
"Aquatic Animals, Others",2769,10,"Fish, seafood",10.7
"Fish, Body Oil",2781,11,Non-food,0.0
"Fish, Liver Oil",2782,11,Non-food,0.0


In [6]:
# Saving dataframe to csv
food_info.to_csv('../../data/food/food_item_info.csv', sep=':', index_label='name')